In [ ]:
%reload_ext autoreload
%autoreload 3
import os

import xarray as xr
from pathlib import Path
from tqdm.auto import tqdm
import torch
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder

from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from dvclive.lightning import DVCLiveLogger

from src.data.julia2018 import Julia2018DataModule
from src.acnets import ACNets


# force running in the project directory (e.g., via nbconvert)
if 'notebooks' in os.getcwd():
    %cd ..

In [ ]:
datamodule = Julia2018DataModule(segment_size=31, split_on='sequence', batch_size=96, shuffle=False)
datamodule.prepare_data()
n_subjects = len(datamodule.subjects)
n_inputs = datamodule.n_features

In [ ]:

n_features = 32
hidden_size = n_features

model = ACNets(n_inputs, n_features, hidden_size, n_subjects)
# model = torch.compile(model)  # is not supported on GTX 1070

trainer = pl.Trainer(
    max_epochs=100, accelerator='auto', log_every_n_steps=1,
    logger=[DVCLiveLogger(save_dvc_exp=True, report='md'),
            TensorBoardLogger('.')]
    )
trainer.fit(model, datamodule=datamodule)
